<a href="https://colab.research.google.com/github/nikodemadamski/FYP/blob/main/FYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

## importing

importing the data sets directly from github along side multiple handy imports i will to use in the future.

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import requests
import io
import re

In [2]:
url = 'https://raw.githubusercontent.com/nikodemadamski/FYP/main/celebrities%20dataset.csv'
url2 = 'https://raw.githubusercontent.com/nikodemadamski/FYP/main/tiktok_1(1).csv'
download = requests.get(url).content
download2 = requests.get(url2).content


df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df2 = pd.read_csv(io.StringIO(download2.decode('utf-8')))

### testing

In [3]:
df = df.dropna()

In [4]:
df.columns

Index(['Month', 'Charli D'Amelio web', 'Charli D'Amelio yt',
       'Charli D'Amelio image', 'Charli D'Amelio news',
       'Charli D'Amelio Total', 'Jules LeBlanc web', 'Jules LeBlanc yt',
       'Jules LeBlanc image', 'Jules LeBlanc news',
       ...
       'Cole Sprouse web', 'Cole Sprouse yt', 'Cole Sprouse image',
       'Cole Sprouse news', 'Cole Sprouse Total', 'Joe Biden web',
       'Joe Biden yt', 'Joe Biden image', 'Joe Biden news', 'Joe Biden Total'],
      dtype='object', length=246)

In [5]:
print(df2.head(100))

               Name  Following Followers  Likes  Views      V_Date V_Likes  \
0   charlidamelio         1267    136.7M  10.6B   1.4M     3h ago   342.1K   
1   charlidamelio         1267    136.7M  10.6B   4.4M     2d ago   799.3K   
2   charlidamelio         1267    136.7M  10.6B    27M     4d ago     5.8M   
3   charlidamelio         1267    136.7M  10.6B   9.8M     4d ago     1.5M   
4   charlidamelio         1267    136.7M  10.6B   6.5M     4d ago   828.5K   
..              ...        ...       ...    ...    ...         ...     ...   
95  charlidamelio         1267    136.7M  10.6B  18.8M  2021-12-10    2.4M   
96  charlidamelio         1267    136.7M  10.6B  16.2M  2021-12-10    2.1M   
97  charlidamelio         1267    136.7M  10.6B    18M   2021-12-9    1.4M   
98  charlidamelio         1267    136.7M  10.6B    19M   2021-12-9    2.7M   
99  charlidamelio         1267    136.7M  10.6B  17.2M   2021-12-9      2M   

   V_Comments  
0         19K  
1       55.6K  
2       83.1K  

In [6]:
df2.dtypes

Name          object
Following      int64
Followers     object
Likes         object
Views         object
V_Date        object
V_Likes       object
V_Comments    object
dtype: object

## EDA1

I have to now change data set 2 , this will be in 2 ways, 1 will eliminate all data with V_date from after 2021, Second will average the videos from daily videos to monthly videos averages to be able to correlate with the data from the first data set

### 1

 eliminating all data after 2021
 [Reference](https://www.statology.org/pandas-sum-column-with-condition/)

In [7]:
df2 = df2[df2["V_Date"].str.contains("ago")==False]

In [8]:
df2 = df2[df2['V_Date'].between('2016-01-01', '2021-31-01')]

### 1.5
 Changing B into Billions, M into million and K into Thousands
 And Changing notation on values that are too large e.g 10.6B from Float to int as none of the large values have an exact amount so they are rounded to the nearest Billion,Million or thousand
  [Reference](https://stackoverflow.com/questions/39684548/convert-the-string-2-90k-to-2900-or-5-2m-to-5200000-in-pandas-dataframe)
  [Reference2](https://stackoverflow.com/questions/658763/how-to-suppress-scientific-notation-when-printing-float-values)

In [9]:
df2.Views = (df2.Views.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.Views.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [10]:
df2.V_Likes = (df2.V_Likes.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.V_Likes.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [11]:
df2.V_Comments = (df2.V_Comments.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.V_Comments.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [12]:
df2.Likes = (df2.Likes.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.Likes.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [13]:
df2.Followers = (df2.Followers.replace(r'[KMB]+$', '', regex=True).astype(float) * \
                   df2.Followers.str.extract(r'[\d\.]+([KMB]+)', expand=False)
                   .fillna(1)
                   .replace(['K','M','B'], [10**3, 10**6,10**9]).astype(int))

In [14]:
df2.head()

,Name,Following,Followers,Likes,Views,V_Date,V_Likes,V_Comments
70,charlidamelio,1267,136700000.0,1.060000e+10,16800000.0,2021-12-31,2100000.0,43600.0
71,charlidamelio,1267,136700000.0,1.060000e+10,15200000.0,2021-12-29,1000000.0,24400.0
72,charlidamelio,1267,136700000.0,1.060000e+10,12000000.0,2021-12-28,693300.0,15800.0
73,charlidamelio,1267,136700000.0,1.060000e+10,13400000.0,2021-12-28,859300.0,22200.0
74,charlidamelio,1267,136700000.0,1.060000e+10,21400000.0,2021-12-25,2400000.0,38400.0


In [15]:
# repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9', }
# df2['Views'].replace(repl_dict, regex=True).map(pd.eval)

In [16]:
tList = ['Following','Followers','Likes','Views','V_Likes','V_Comments']
for x in tList:
  df2[x]=df2[x].astype(int)

In [17]:
df2.dtypes

Name          object
Following      int64
Followers      int64
Likes          int64
Views          int64
V_Date        object
V_Likes        int64
V_Comments     int64
dtype: object

In [18]:
df2.head()

,Name,Following,Followers,Likes,Views,V_Date,V_Likes,V_Comments
70,charlidamelio,1267,136700000,10600000000,16800000,2021-12-31,2100000,43600
71,charlidamelio,1267,136700000,10600000000,15200000,2021-12-29,1000000,24400
72,charlidamelio,1267,136700000,10600000000,12000000,2021-12-28,693300,15800
73,charlidamelio,1267,136700000,10600000000,13400000,2021-12-28,859300,22200
74,charlidamelio,1267,136700000,10600000000,21400000,2021-12-25,2400000,38400


### 2

averaging into monthly videos

first lets change the V_date from a string into a valid time

In [19]:
df2["V_Date"] = pd.to_datetime(df2["V_Date"])

next lets try to group by time
  [Reference](https://stackoverflow.com/questions/65471540/get-monthly-average-in-pandas)

In [20]:
df3 = df2.groupby(['Name',pd.PeriodIndex(df2['V_Date'],
                                          freq="M")]).mean()

In [21]:
tList = ['Following','Followers','Likes','Views','V_Likes','V_Comments']
for x in tList:
  df3[x]=df3[x].astype(int)

In [22]:
df3.dtypes

Following     int64
Followers     int64
Likes         int64
Views         int64
V_Likes       int64
V_Comments    int64
dtype: object

In [23]:
tList = ['Views','V_Likes','V_Comments']
for x in tList:
  df3[x]=df3[x].astype(int)

In [24]:
df3

Following  Followers       Likes     Views  V_Likes  \
Name          V_Date                                                         
addisonre     2019-07       2337   86700000  5600000000   8431578  1112921   
              2019-08       2337   86700000  5600000000   5662500   781181   
              2019-09       2337   86700000  5600000000   4987500   655275   
              2019-10       2337   86700000  5600000000   5142857   708554   
              2019-11       2337   86700000  5600000000  12794827  1522032   
...                          ...        ...         ...       ...      ...   
taylorswift   2021-10          0   11000000   100500000  12100000  2100000   
              2021-11          0   11000000   100500000  25190909  4609090   
              2021-12          0   11000000   100500000   8000000  1900000   
tommyinnit    2021-11         45   11300000   246000000   3666666   828966   
              2021-12         45   11300000   246000000   4033333   774850   

                       V_Comments  
Name          V_Date               
addisonre     2019-07       21930  
              2019-08        6234  
              2019-09        6045  
              2019-10        4447  
              2019-11        6892  
...                           ...  
taylorswift   2021-10       35650  
              2021-11       56609  
              2021-12       32600  
tommyinnit    2021-11       27233  
              2021-12       20850  

[571 rows x 6 columns]

##EDA2

divide the df into a 2nd dataset which will combine all totals into people who do not have tiktok which will be my control data and people who do have it,
further analysis will continue by dividing the celebrities up into more groups for a more in debth analysis

  [Reference](https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe)


List of celebrities who dont have TikTok will be my control group:\
Pewdiepie\
Millie Bobby Brown\
Kim - no post\
Xxxtentacion\
Tom Holland\
Donald Trump\
Beyonce - no post\
Michael Jackson\
Coco Quinn\
Kobe Bryant\
Hayley LeBlanc\
Txunamy\
Zendaya\
Emma Chamberlain\
Cole Sprouse\
Joe Biden

In [25]:
n_list2=['web','yt','image','news']
df_total = df
for x in n_list2:
  df_total = df_total[df_total.columns.drop(list(df_total.filter(regex=x)))]

In [26]:
no_tt_cel = ['Pewdiepie',
'Millie Bobby Brown',
'Kim Kardashian',
'XXXTENTACION',
'Tom Holland',
'Donald Trump',
'Beyonce',
'Michael Jackson',
'Coco Quinn',
'Kobe Bryant',
'Hayley LeBlanc',
'Txunamy',
'Zendaya',
'Emma Chamberlain',
'Cole Sprouse',
'Joe Biden']

next lets try to move all the users who dont have tiktok into a seperate column and remove their previous data.

In [27]:
df_total2 = df_total
for x in no_tt_cel:
  df_total2 = df_total2[df_total2.columns.drop(list(df_total2.filter(regex=x)))]

In [28]:
df_total2

,Month,Charli D'Amelio Total,Jules Leblanc total,Addison Rae Total,Ariana Grande Total,Justin Bieber Total,Piper Rockelle Total,Jacob Satorious Total,Dixie D'Amelio Total,Lil Huddy Total,...,Baby Ariel Total,Nicki Minaj Total,Josh Riochard Total,Cash Baker Total,Noah Beck Total,Tony Lopez Total,Jake Paul Total,Brent Rivera Total,David Dobrik Total,Gavin Magnus Total
0,2016-01,3.50,5.25,0.500000,63.75,99.25,4.000000,18.50,2.50,0.000000,...,41.75,80.25,8.75,0.00,0.00,1.000000,3.500000,14.50,1.333333,0.00
1,2016-02,3.00,4.75,0.333333,50.50,84.75,3.666667,37.00,1.00,0.000000,...,29.50,68.75,7.25,0.00,0.00,0.500000,2.666667,14.00,2.000000,0.00
2,2016-03,2.50,6.50,1.000000,65.50,76.00,2.750000,38.50,1.00,0.333333,...,37.50,62.25,39.50,0.00,0.00,0.333333,2.333333,15.75,2.750000,0.00
3,2016-04,2.50,5.50,0.000000,61.50,67.75,6.333333,46.75,1.00,0.000000,...,66.50,62.50,40.25,0.00,0.00,0.750000,3.500000,14.25,2.750000,0.00
4,2016-05,2.25,8.50,0.333333,68.00,65.50,4.000000,85.50,1.00,0.000000,...,78.50,58.00,17.50,0.00,0.00,1.500000,3.500000,14.00,2.500000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2021-08,32.50,12.25,40.500000,31.00,13.75,68.750000,2.50,27.75,20.500000,...,6.50,20.75,20.75,1.75,16.75,6.000000,55.250000,52.25,20.500000,47.75
68,2021-09,30.00,11.50,37.500000,26.75,14.50,82.750000,2.00,30.25,13.000000,...,5.25,29.00,2.50,1.25,12.00,5.000000,11.250000,41.75,19.750000,34.75
69,2021-10,21.75,12.25,22.750000,23.75,11.75,48.750000,2.00,13.50,9.500000,...,4.25,20.00,7.75,2.50,13.00,3.000000,8.750000,43.50,16.000000,28.50
70,2021-11,23.00,13.25,21.250000,23.00,14.00,51.750000,2.25,13.00,8.750000,...,4.50,17.00,6.25,1.75,14.50,3.500000,10.750000,49.75,15.000000,37.50


In [29]:
df_total3 = df_total[['Month','PewDiePie Total', 'Millie Bobby Brown Total','Kim Kardashian total','XXXTENTACION Total','Tom Holland Total','Donald Trump Total','Michael Jackson Total','Beyonce Total','Coco Quinn Total','Kobe Bryant Total', 'Hayley LeBlanc Total','Txunamy Total', 'Zendaya Total', 'Emma Chamberlain Total',
       'Cole Sprouse Total', 'Joe Biden Total']]

In [30]:
df_Test = df_total3.copy()
df_Test['Control'] = df_Test.median(numeric_only=True, axis=1)
df_Test

,Month,PewDiePie Total,Millie Bobby Brown Total,Kim Kardashian total,XXXTENTACION Total,Tom Holland Total,Donald Trump Total,Michael Jackson Total,Beyonce Total,Coco Quinn Total,Kobe Bryant Total,Hayley LeBlanc Total,Txunamy Total,Zendaya Total,Emma Chamberlain Total,Cole Sprouse Total,Joe Biden Total,Control
0,2016-01,36.00,0.00,49.25,0.00,2.50,15.25,57.25,45.75,0.00,3.500000,3.00,0.75,24.25,0.00,10.50,1,3.50
1,2016-02,34.00,1.00,49.75,0.00,2.00,21.25,55.75,99.00,0.00,4.500000,1.50,4.75,32.25,0.00,11.50,1,4.75
2,2016-03,30.75,1.00,76.75,0.00,7.25,34.25,52.00,47.75,1.50,3.250000,1.50,1.50,27.25,0.00,11.75,1,7.25
3,2016-04,35.75,0.00,50.50,0.00,5.50,21.75,58.75,86.75,0.00,12.750000,1.50,6.50,23.75,0.00,9.75,1,9.75
4,2016-05,39.75,0.00,46.75,0.00,7.50,22.25,53.00,72.50,0.00,3.250000,1.50,3.00,20.50,0.00,9.25,1,7.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2021-08,10.00,28.75,17.75,8.50,20.00,5.00,29.50,12.75,40.75,2.750000,24.75,68.50,24.75,27.50,25.25,15,24.75
68,2021-09,8.75,25.00,26.50,8.75,21.25,4.75,30.00,12.25,30.50,2.666667,11.75,58.50,33.25,74.25,17.75,13.25,21.25
69,2021-10,8.50,20.00,26.00,9.25,19.00,4.25,33.75,10.75,21.25,2.666667,8.75,32.25,29.25,39.75,16.00,10.5,19.00
70,2021-11,8.25,20.50,27.00,9.25,29.00,4.00,31.75,10.25,18.75,2.250000,18.50,34.75,28.50,27.00,15.25,11.75,18.75


In [31]:
Test = df_Test[['Month','Control']]
Test

,Month,Control
0,2016-01,3.50
1,2016-02,4.75
2,2016-03,7.25
3,2016-04,9.75
4,2016-05,7.50
...,...,...
67,2021-08,24.75
68,2021-09,21.25
69,2021-10,19.00
70,2021-11,18.75


Next lets add 2 more columns into test, new tiktok users and old tiktok users

first lets fix name and V_Date as they are not officially column names

In [32]:
df2.groupby(["Name"]).agg({'V_Date' : np.min})

,V_Date
Name,
addisonre,2019-07-09
arianagrande,2020-10-25
avani,2021-10-01
babyariel,2021-10-12
billieeilish,2020-11-12
brentrivera,2021-12-14
brycehall,2019-08-01
camerondallas,2019-03-27
cashbaker,2021-12-31


In [33]:
df_total.columns

Index(['Month', 'Charli D'Amelio Total', 'Jules Leblanc total',
       'Addison Rae Total', 'Ariana Grande Total', 'Justin Bieber Total',
       'Piper Rockelle Total', 'Jacob Satorious Total', 'Dixie D'Amelio Total',
       'Lil Huddy Total', 'JoJo Siwa Total', 'Maddie Ziegler Total',
       'Mickenzie Ziegler Total', 'James Charles Total', 'PewDiePie Total',
       'Loren Grey Total', 'Millie Bobby Brown Total', 'Billie Eilish total',
       'Kylie Jenner Total', 'Selena Gomez Total', 'Taylor Swift Total',
       'Kim Kardashian total', 'Cardi B Total', 'Shawn Mendes Total',
       'Bryce Hall Total', 'XXXTENTACION Total', 'Avani Gregg Total',
       'Olicia Rodrigo Total', 'Tom Holland Total', 'Donald Trump Total',
       'Baby Ariel Total', 'Nicki Minaj Total', 'Josh Riochard Total',
       'Beyonce Total', 'Michael Jackson Total', 'Cash Baker Total',
       'Noah Beck Total', 'Tony Lopez Total', 'Jake Paul Total',
       'Brent Rivera Total', 'David Dobrik Total', 'Coco Quinn Tota

In [34]:
Early = df_total[['Month','JoJo Siwa Total','Jacob Satorious Total','Lil Huddy Total','Loren Grey Total','Selena Gomez Total','Cardi B Total',
                  "Charli D'Amelio Total",'Addison Rae Total', 'Bryce Hall Total','James Charles Total', 'Jules Leblanc total',"Dixie D'Amelio Total",
                  'Mickenzie Ziegler Total','Shawn Mendes Total','Justin Bieber Total',]]
late = df_total[['Month','Ariana Grande Total','Piper Rockelle Total', 'Maddie Ziegler Total','Loren Grey Total','Billie Eilish total','Taylor Swift Total','Avani Gregg Total',
                 'Olicia Rodrigo Total','Baby Ariel Total','Nicki Minaj Total', 'Josh Riochard Total','Cash Baker Total','Brent Rivera Total','Hayley LeBlanc Total', 'Gavin Magnus Total']]

In [35]:
Early['Early TikToker'] = Early.median(numeric_only=True, axis=1)
late['late TikToker'] = late.median(numeric_only=True, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
Test4 = Early[['Month','Early TikToker']]
extracted_col9 = late["late TikToker"]
Test4.insert(2, "late TikToker", extracted_col9)
extracted_col = Test["Control"]
Test4.insert(1, "Control", extracted_col)

In [37]:
Test4

,Month,Control,Early TikToker,late TikToker
0,2016-01,3.50,4.25,14.50
1,2016-02,4.75,4.75,14.00
2,2016-03,7.25,5.00,22.75
3,2016-04,9.75,4.00,38.75
4,2016-05,7.50,8.50,17.50
...,...,...,...,...
67,2021-08,24.75,20.00,22.50
68,2021-09,21.25,18.00,19.75
69,2021-10,19.00,13.50,15.75
70,2021-11,18.75,16.50,18.50


In [38]:
df3.columns

Index(['Following', 'Followers', 'Likes', 'Views', 'V_Likes', 'V_Comments'], dtype='object')

In [39]:
df3

Following  Followers       Likes     Views  V_Likes  \
Name          V_Date                                                         
addisonre     2019-07       2337   86700000  5600000000   8431578  1112921   
              2019-08       2337   86700000  5600000000   5662500   781181   
              2019-09       2337   86700000  5600000000   4987500   655275   
              2019-10       2337   86700000  5600000000   5142857   708554   
              2019-11       2337   86700000  5600000000  12794827  1522032   
...                          ...        ...         ...       ...      ...   
taylorswift   2021-10          0   11000000   100500000  12100000  2100000   
              2021-11          0   11000000   100500000  25190909  4609090   
              2021-12          0   11000000   100500000   8000000  1900000   
tommyinnit    2021-11         45   11300000   246000000   3666666   828966   
              2021-12         45   11300000   246000000   4033333   774850   

                       V_Comments  
Name          V_Date               
addisonre     2019-07       21930  
              2019-08        6234  
              2019-09        6045  
              2019-10        4447  
              2019-11        6892  
...                           ...  
taylorswift   2021-10       35650  
              2021-11       56609  
              2021-12       32600  
tommyinnit    2021-11       27233  
              2021-12       20850  

[571 rows x 6 columns]

i cant figure it out, for now lets do 2 groups, tiktok and control

In [40]:
df_Test2 = df_total2.copy()
df_Test2['TikTok'] = df_Test2.median(numeric_only=True, axis=1)
df_Test2

,Month,Charli D'Amelio Total,Jules Leblanc total,Addison Rae Total,Ariana Grande Total,Justin Bieber Total,Piper Rockelle Total,Jacob Satorious Total,Dixie D'Amelio Total,Lil Huddy Total,...,Nicki Minaj Total,Josh Riochard Total,Cash Baker Total,Noah Beck Total,Tony Lopez Total,Jake Paul Total,Brent Rivera Total,David Dobrik Total,Gavin Magnus Total,TikTok
0,2016-01,3.50,5.25,0.500000,63.75,99.25,4.000000,18.50,2.50,0.000000,...,80.25,8.75,0.00,0.00,1.000000,3.500000,14.50,1.333333,0.00,4.750000
1,2016-02,3.00,4.75,0.333333,50.50,84.75,3.666667,37.00,1.00,0.000000,...,68.75,7.25,0.00,0.00,0.500000,2.666667,14.00,2.000000,0.00,4.750000
2,2016-03,2.50,6.50,1.000000,65.50,76.00,2.750000,38.50,1.00,0.333333,...,62.25,39.50,0.00,0.00,0.333333,2.333333,15.75,2.750000,0.00,5.750000
3,2016-04,2.50,5.50,0.000000,61.50,67.75,6.333333,46.75,1.00,0.000000,...,62.50,40.25,0.00,0.00,0.750000,3.500000,14.25,2.750000,0.00,5.916667
4,2016-05,2.25,8.50,0.333333,68.00,65.50,4.000000,85.50,1.00,0.000000,...,58.00,17.50,0.00,0.00,1.500000,3.500000,14.00,2.500000,0.00,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2021-08,32.50,12.25,40.500000,31.00,13.75,68.750000,2.50,27.75,20.500000,...,20.75,20.75,1.75,16.75,6.000000,55.250000,52.25,20.500000,47.75,20.625000
68,2021-09,30.00,11.50,37.500000,26.75,14.50,82.750000,2.00,30.25,13.000000,...,29.00,2.50,1.25,12.00,5.000000,11.250000,41.75,19.750000,34.75,17.625000
69,2021-10,21.75,12.25,22.750000,23.75,11.75,48.750000,2.00,13.50,9.500000,...,20.00,7.75,2.50,13.00,3.000000,8.750000,43.50,16.000000,28.50,14.625000
70,2021-11,23.00,13.25,21.250000,23.00,14.00,51.750000,2.25,13.00,8.750000,...,17.00,6.25,1.75,14.50,3.500000,10.750000,49.75,15.000000,37.50,15.875000


In [41]:
Test2 = df_Test2[['Month','TikTok']]
Test2

,Month,TikTok
0,2016-01,4.750000
1,2016-02,4.750000
2,2016-03,5.750000
3,2016-04,5.916667
4,2016-05,10.000000
...,...,...
67,2021-08,20.625000
68,2021-09,17.625000
69,2021-10,14.625000
70,2021-11,15.875000


In [42]:
extracted_col = Test2["TikTok"]
Test.insert(2, "TikTok", extracted_col)

In [43]:
Test

,Month,Control,TikTok
0,2016-01,3.50,4.750000
1,2016-02,4.75,4.750000
2,2016-03,7.25,5.750000
3,2016-04,9.75,5.916667
4,2016-05,7.50,10.000000
...,...,...,...
67,2021-08,24.75,20.625000
68,2021-09,21.25,17.625000
69,2021-10,19.00,14.625000
70,2021-11,18.75,15.875000


**tiktoker**: 'Charli D'Amelio Total','Addison Rae Total','Dixie D'Amelio Total','Lil Huddy Total','JoJo Siwa Total','Maddie Ziegler Total','Mickenzie Ziegler Total','Loren Grey Total','Bryce Hall Total','Avani Gregg Total','Tom Holland Total','Baby Ariel Total','Josh Riochard Total','Cash Baker Total','Noah Beck Total', 'Tony Lopez Total','Coco Quinn Total','Txunamy Total'\
**singer**: 'Ariana Grande Total','Justin Bieber Total', 'Piper Rockelle Total','Billie Eilish total','Taylor Swift Total','Cardi B Total', 'Shawn Mendes Total','XXXTENTACION Total', 'Olicia Rodrigo Total','Nicki Minaj Total','Beyonce Total', 'Michael Jackson Total'\
**Youtuber**: 'Jules Leblanc total','Jacob Satorious Total', 'James Charles Total', 'PewDiePie Total','Jake Paul Total','Brent Rivera Total', 'David Dobrik Total','Gavin Magnus Total','Emma Chamberlain Total'\
**Actor**: 'Millie Bobby Brown Total','Selena Gomez Total','Hayley LeBlanc Total','Zendaya Total','Cole Sprouse Total'\
**Other**: 'Kylie Jenner Total','Kim Kardashian total','Donald Trump Total','Kobe Bryant Total','Joe Biden Total'

In [44]:
TikTokers = df_total[['Month',"Charli D'Amelio Total",'Addison Rae Total',"Dixie D'Amelio Total",'Lil Huddy Total','JoJo Siwa Total','Maddie Ziegler Total','Mickenzie Ziegler Total','Loren Grey Total','Bryce Hall Total','Avani Gregg Total','Tom Holland Total','Baby Ariel Total','Josh Riochard Total',
                      'Cash Baker Total','Noah Beck Total', 'Tony Lopez Total','Coco Quinn Total','Txunamy Total']]

In [45]:
df_Test3 = TikTokers.copy()
df_Test3['TikTokers Total'] = df_Test3.median(numeric_only=True, axis=1)

In [46]:
Singer = df_total[['Month','Ariana Grande Total','Justin Bieber Total', 'Piper Rockelle Total','Billie Eilish total','Taylor Swift Total','Cardi B Total', 'Shawn Mendes Total','XXXTENTACION Total',
                     'Olicia Rodrigo Total','Nicki Minaj Total','Beyonce Total', 'Michael Jackson Total']]

In [47]:
df_Test4 = Singer.copy()
df_Test4['Singer Total'] = df_Test4.median(numeric_only=True, axis=1)

In [48]:
Youtuber = df_total[['Month','Jules Leblanc total','Jacob Satorious Total', 'James Charles Total', 'PewDiePie Total','Jake Paul Total',
                     'Brent Rivera Total', 'David Dobrik Total','Gavin Magnus Total','Emma Chamberlain Total']]

In [49]:
df_Test5 = Youtuber.copy()
df_Test5['Youtuber Total'] = df_Test5.median(numeric_only=True, axis=1)

In [50]:
Actors = df_total[['Month','Millie Bobby Brown Total','Selena Gomez Total','Hayley LeBlanc Total','Zendaya Total','Cole Sprouse Total']]

In [51]:
df_Test6 = Actors.copy()
df_Test6['Actors Total'] = df_Test6.median(numeric_only=True, axis=1)

In [52]:
Others = df_total[['Month','Kylie Jenner Total','Kim Kardashian total','Donald Trump Total','Kobe Bryant Total','Joe Biden Total']]

In [53]:
df_Test7 = Others.copy()
df_Test7['Others Total'] = df_Test7.median(numeric_only=True, axis=1)

In [54]:
Test3 = df_Test3[['Month','TikTokers Total']]
Test3

,Month,TikTokers Total
0,2016-01,2.500000
1,2016-02,2.000000
2,2016-03,1.500000
3,2016-04,1.916667
4,2016-05,2.458333
...,...,...
67,2021-08,22.250000
68,2021-09,18.500000
69,2021-10,13.500000
70,2021-11,15.875000


In [55]:
extracted_col2 = df_Test4["Singer Total"]
extracted_col3 = df_Test5["Youtuber Total"]
extracted_col4 = df_Test6["Actors Total"]
extracted_col5 = df_Test7["Others Total"]
Test3.insert(2, "Singer Total", extracted_col2)
Test3.insert(3, "Youtuber Total", extracted_col3)
Test3.insert(4, "Actors Total", extracted_col4)
Test3.insert(5, "Others Total", extracted_col5)

In [56]:
Test3

,Month,TikTokers Total,Singer Total,Youtuber Total,Actors Total,Others Total
0,2016-01,2.500000,57.750,3.500000,10.50,32.250
1,2016-02,2.000000,54.000,2.666667,11.50,34.000
2,2016-03,1.500000,49.875,2.750000,11.75,39.250
3,2016-04,1.916667,58.250,3.500000,9.75,36.125
4,2016-05,2.458333,50.625,3.500000,9.25,34.500
...,...,...,...,...,...,...
67,2021-08,22.250000,17.875,20.500000,24.75,11.375
68,2021-09,18.500000,19.625,11.500000,19.00,14.500
69,2021-10,13.500000,16.750,12.250000,20.00,10.875
70,2021-11,15.875000,20.000,13.250000,18.50,13.000


# HELP

df3 have column 'Name' & 'V_date' But they are not usable

In [57]:
df3

Following  Followers       Likes     Views  V_Likes  \
Name          V_Date                                                         
addisonre     2019-07       2337   86700000  5600000000   8431578  1112921   
              2019-08       2337   86700000  5600000000   5662500   781181   
              2019-09       2337   86700000  5600000000   4987500   655275   
              2019-10       2337   86700000  5600000000   5142857   708554   
              2019-11       2337   86700000  5600000000  12794827  1522032   
...                          ...        ...         ...       ...      ...   
taylorswift   2021-10          0   11000000   100500000  12100000  2100000   
              2021-11          0   11000000   100500000  25190909  4609090   
              2021-12          0   11000000   100500000   8000000  1900000   
tommyinnit    2021-11         45   11300000   246000000   3666666   828966   
              2021-12         45   11300000   246000000   4033333   774850   

                       V_Comments  
Name          V_Date               
addisonre     2019-07       21930  
              2019-08        6234  
              2019-09        6045  
              2019-10        4447  
              2019-11        6892  
...                           ...  
taylorswift   2021-10       35650  
              2021-11       56609  
              2021-12       32600  
tommyinnit    2021-11       27233  
              2021-12       20850  

[571 rows x 6 columns]

as you can see above 'Name' and 'V_Date' are here but not as a column

In [58]:
df3.columns

Index(['Following', 'Followers', 'Likes', 'Views', 'V_Likes', 'V_Comments'], dtype='object')

so when i try and call a function to check for first date for every name e.g charlie started tiktok on january 2016, this is what occurs

In [60]:
# df3.groupby(["Name"]).agg({'V_Date' : np.min})

but i know this works when i attempted d2 before.

### so what do i do ?

problem will occur when i attempt to compare this df3 dataframe to another dataframe as name and V_date will be invalid column names i could use.
thank you in advance